In [1]:
#在Colab用
!pip install -Uq timm==0.9.12


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 104.4 MB/s eta 0:00:00


In [2]:
#授權
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.cuda.amp import GradScaler, autocast
import torch.nn.functional as F
import timm

In [4]:
num_epochs = 50
batch_size = 16
early_stop_patience = 4
patience_counter = 0
loss_alpha=1.0
loss_beta=0.5

In [5]:
base_dir = "/content/drive/MyDrive/DLA_term_project_data/classified_data"

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
imagenet_stats = [(0.485, 0.456, 0.406), (0.229, 0.224, 0.225)]
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_stats[0], imagenet_stats[1])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

validation_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_stats[0], imagenet_stats[1])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(imagenet_stats[0], imagenet_stats[1])
])

train_dataset = datasets.ImageFolder(os.path.join(base_dir, 'train'), transform=train_transform)
val_dataset = datasets.ImageFolder(os.path.join(base_dir, 'validation'), transform=validation_transform)
test_data = datasets.ImageFolder(os.path.join(base_dir, "test"), transform=test_transform)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print("類別數量：", len(train_dataset.classes))

類別數量： 22


In [6]:
# customed loss function
# ----- 1. Node-to-Node Distance Matrix -----
node_distance_matrix = torch.tensor([
    [0,1,2,2,1,2,2,2.5,3.5,4,5.5,5,4,2.5,3.5,4,5,3.5,4.5,3.5,2.5,2.5],
    [1,0,1,3,2,3,1,1.5,2.5,3,4.5,4,3,1.5,4.5,5,6,4.5,5.5,4.5,3.5,3.5],
    [2,1,0,2,3,4,2,2.5,3.5,4,5.5,3,2,0.5,3.5,4,5,3.5,4.5,5.5,4.5,2.5],
    [2,3,2,0,1,2,4,4.5,5.5,6,7.5,5,4,2.5,1.5,2,3,1.5,2.5,3.5,2.5,0.5],
    [1,2,3,1,0,1,3,3.5,4.5,5,6.5,6,5,3.5,2.5,3,4,2.5,3.5,2.5,1.5,1.5],
    [2,3,4,2,1,0,2,2.5,3.5,4,5.5,7,6,4.5,3.5,4,5,3.5,2.5,1.5,0.5,2.5],
    [2,1,2,4,3,2,0,0.5,1.5,2,3.5,5,4,2.5,5.5,6,7,5.5,4.5,3.5,2.5,4.5],
    [2.5,1.5,2.5,4.5,3.5,2.5,0.5,0,1,1.5,3,3.5,4.5,3,6,6.5,7.5,6,5,4,3,5],
    [3.5,2.5,3.5,5.5,4.5,3.5,1.5,1,0,0.5,2,2.5,3.5,4,7,7.5,8.5,7,6,5,4,6],
    [4,3,4,6,5,4,2,1.5,0.5,0,1.5,2,3,4.5,7.5,8,9,7.5,6.5,5.5,4.5,6.5],
    [5.5,4.5,5.5,7.5,6.5,5.5,3.5,3,2,1.5,0,0.5,1.5,3,7,7.5,8.5,7,8,7,6,6.5],
    [5,4,3,5,6,7,5,3.5,2.5,2,0.5,0,1,2.5,6.5,7,8,6.5,7.5,7.5,6.5,5.5],
    [4,3,2,4,5,6,4,4.5,3.5,3,1.5,1,0,1.5,5.5,6,7,5.5,6.5,7.5,7,4.5],
    [2.5,1.5,0.5,2.5,3.5,4.5,2.5,3,4,4.5,3,2.5,1.5,0,4,4.5,5.5,4,5,6,5.5,3],
    [3.5,4.5,3.5,1.5,2.5,3.5,5.5,6,7,7.5,7,6.5,5.5,4,0,0.5,1.5,0.5,1.5,2.5,3.5,1],
    [4,5,4,2,3,4,6,6.5,7.5,8,7.5,7,6,4.5,0.5,0,1,1,2,3,4,1.5],
    [5,6,5,3,4,5,7,7.5,8.5,9,8.5,8,7,5.5,1.5,1,0,1.5,2.5,3.5,4.5,2.5],
    [3.5,4.5,3.5,1.5,2.5,3.5,5.5,6,7,7.5,7,6.5,5.5,4,0.5,1,1.5,0,1,2,3,1],
    [4.5,5.5,4.5,2.5,3.5,2.5,4.5,5,6,6.5,8,7.5,6.5,5,1.5,2,2.5,1,0,1,2,2],
    [3.5,4.5,5.5,3.5,2.5,1.5,3.5,4,5,5.5,7,7.5,7.5,6,2.5,3,3.5,2,1,0,1,3],
    [2.5,3.5,4.5,2.5,1.5,0.5,2.5,3,4,4.5,6,6.5,7,5.5,3.5,4,4.5,3,2,1,0,3],
    [2.5,3.5,2.5,0.5,1.5,2.5,4.5,5,6,6.5,6.5,5.5,4.5,3,1,1.5,2.5,1,2,3,3,0]
], dtype=torch.float)

# ----- 2. Area-to-Area Distance Matrix -----
area_distance_matrix = torch.tensor([
    [0, 1, 2, 2, 2, 2],
    [1, 0, 1, 1, 1, 1],
    [2, 1, 0, 1, 3, 3],
    [2, 1, 1, 0, 3, 3],
    [2, 1, 3, 3, 0, 1],
    [2, 1, 3, 3, 1, 0]
], dtype=torch.float)

# ----- 3. Node-to-Area Mapping -----
node_to_area = torch.tensor([
    0, 0, 0, 0, 0, 0, 0,    # 1~7 → area 0
    1,                      # 8  → area 1
    2, 2,                   # 9~10 → area 2
    3, 3, 3,                # 11~13 → area 3
    1,                      # 14 → area 1
    4, 4, 4,                # 15~17 → area 4
    5, 5, 5,                # 18~20 → area 5
    1,                      # 21 → area 1
    1                       # 22 → area 1
], dtype=torch.long)

# ----- 4. Custom Loss Function -----
class DistancePenaltyLoss(nn.Module):
    def __init__(self, node_distance_matrix, area_distance_matrix, node_to_area, alpha=1.0, beta=1.0):
        super().__init__()
        self.node_distance_matrix = node_distance_matrix
        self.area_distance_matrix = area_distance_matrix
        self.node_to_area = node_to_area
        self.alpha = alpha
        self.beta = beta
        self.ce = nn.CrossEntropyLoss()

    def forward(self, logits, targets):
        """
        logits: (batch_size, 22)
        targets: (batch_size,) - ground truth class indices
        """
        ce_loss = self.ce(logits, targets)

        probs = F.softmax(logits, dim=1)  # (B, 22)
        batch_size = targets.size(0)

        node_penalty = 0.0
        area_penalty = 0.0

        for i in range(batch_size):
            target = targets[i]
            prob = probs[i]  # shape (22,)

            # Node distance penalty
            node_distances = self.node_distance_matrix[target]  # (22,)
            node_penalty += (prob * node_distances).sum()

            # Area distance penalty
            target_area = self.node_to_area[target]
            pred_areas = self.node_to_area  # shape (22,)
            area_dists = self.area_distance_matrix[target_area][pred_areas]  # shape (22,)
            area_penalty += (prob * area_dists).sum()

        node_penalty /= batch_size
        area_penalty /= batch_size

        total_loss = ce_loss + self.alpha * node_penalty + self.beta * area_penalty
        return total_loss

In [7]:
# timm.list_models('convnext*')
# timm.list_models('convnext*', pretrained=True)
# timm.list_models('efficient*', pretrained=True)
# timm.list_models('res*', pretrained=True)
# timm.list_models('swin*', pretrained=True)

# convnextv2_tiny.fcmae
# efficientvit_b0.r224_in1k
# efficientvit_b1.r224_in1k
# efficientvit_b2.r224_in1k
# efficientvit_b3.r224_in1k
# efficientvit_m0.r224_in1k
# efficientvit_m1.r224_in1k
# efficientvit_m2.r224_in1k
# efficientvit_m3.r224_in1k
# efficientvit_m4.r224_in1k
# efficientvit_m5.r224_in1k
# efficientformerv2_s0.snap_dist_in1k
# efficientformerv2_s1.snap_dist_in1k
# efficientformerv2_s2.snap_dist_in1k
# resnet50.a1_in1k
# resnet50d.ra2_in1k
# resnet50.fb_swsl_ig1b_ft_in1k
# resnet50.tv_in1k
# resnet50.ra_in1k
# res2net50d.in1k
# res2net50_14w_8s.in1k
# mobilevitv2_100.cvnets_in1k
# mobilevitv2_150.cvnets_in22k_ft_in1k
# mobilenetv3_large_100.ra_in1k
# mobileone_s1.apple_in1k
# mobilenetv2_100.ra_in1k
# swin_tiny_patch4_window7_224.ms_in1k
# swinv2_tiny_window8_256.ms_in1k
# swinv2_cr_tiny_ns_224.sw_in1k



# ✅ Overall Best Choices (Balanced: Accuracy, Speed, Size)
# mobilevitv2_100.cvnets_in1k	       Transformer-CNN hybrid, small but very accurate
# efficientnet_b0	             Lightweight, easy to train, great accuracy/size tradeoff
# mobilenetv3_large_100.ra_in1k       Very fast, widely supported, good baseline for small datasets
# res2net50d.in1k	             Strong feature extractor, better than plain ResNet50
# resnet50d.ra2_in1k             Modern ResNet variant with strong augmentations
# swinv2_tiny_window8_256.ms_in1k	     Lightweight transformer, good if you want to try Swin family

# 🧠 If You Want Transfer Learning Strength
# mobilevitv2_150.cvnets_in22k_ft_in1k	  Pretrained on ImageNet-22K → fine-tuned on IN1K
# resnet50.fb_swsl_ig1b_ft_in1k	       Trained on IG-1B dataset — great features even for few-shot

# 🚀 If You Want Ultra-Fast Training
# mobileone_s1.apple_in1k          	Tiny, deployable, optimized for fast inference
# mobilenetv2_100.ra_in1k         	Classic, efficient, and still performs well

In [8]:
# ---------- 100% 手刻 Hybrid CNN + GNN：Patch-GNNNet ----------
import torch
import torch.nn as nn
import torch.nn.functional as F

class PatchGNNNet(nn.Module):
    """
    將輸入影像先透過小型 CNN Stem 提取出一張特徵圖 (C, H, W)，
    然後把特徵圖切成 patch_size×patch_size 的小格，每格當作圖上的一個 Node；
    接著做一次純手刻 GraphConv（以 patch 間的 8 鄰域為鄰接），
    最後做全域平均 + MLP Head 來預測 22 類節點。
    """

    def __init__(self, num_classes=22, in_ch=3, stem_channels=64, patch_size=4, hidden_dim=256, gc_hidden=256):
        """
        Args:
            num_classes: 分類節點數 (22)
            in_ch: 輸入影像通道 (RGB→3)
            stem_channels: CNN Stem 第一層輸出通道數
            patch_size:  把特徵圖切成 patch_size×patch_size 的塊
            hidden_dim: CNN Stem 收尾通道數 (最後會輸出 (hidden_dim, H, W))
            gc_hidden: GNN 隱藏層維度
        """
        super().__init__()
        self.patch_size = patch_size

        # 1) CNN Stem：4 層小型 Conv → BatchNorm → Mish
        #    從 (3×224×224) 下採樣到 (hidden_dim, 56, 56) 以下：
        self.stem = nn.Sequential(
            nn.Conv2d(in_ch, stem_channels, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(stem_channels),
            nn.Mish(),
            nn.MaxPool2d(3, 2, 1),  # -> (stem_channels, 56, 56)

            nn.Conv2d(stem_channels, stem_channels*2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(stem_channels*2),
            nn.Mish(),              # -> (stem_channels*2, 28, 28)

            nn.Conv2d(stem_channels*2, stem_channels*4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(stem_channels*4),
            nn.Mish(),              # -> (stem_channels*4, 14, 14)

            nn.Conv2d(stem_channels*4, hidden_dim, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(hidden_dim),
            nn.Mish()               # -> (hidden_dim, 14, 14)
        )
        # 之所以讓 Stem 最終空間大小保持 (14×14)，是因為 patch_size=4 時 (14 % 4 == 2)
        # 但這裡我們要 (hidden_dim, 56, 56) / 4 -> (hidden_dim, 14, 14)

        # 2) GraphConv 層：單層 GCN-like 更新
        #    對每個 patch node，把自己與鄰居 (8 鄰域) feature 做一次融合
        self.gc_in = nn.Linear(hidden_dim, gc_hidden, bias=False)   # W_self
        self.gc_nei = nn.Linear(hidden_dim, gc_hidden, bias=False)  # W_neigh
        self.gc_act = nn.Mish()

        # 3) MLP Head：把圖卷積後的 patch features 全域平均，接一個隱藏層 + 分類層
        self.mlp_head = nn.Sequential(
            nn.Linear(gc_hidden, gc_hidden),
            nn.Mish(),
            nn.Dropout(0.3),
            nn.Linear(gc_hidden, num_classes)
        )

    def forward(self, x):
        """
        x: (B, 3, 224, 224)  → Stem → (B, hidden_dim, 14, 14)
        → 切 Patch → (B, N, hidden_dim)，其中 N = (14/patch_size) * (14/patch_size)
          (若 patch_size=2，則 N=49；若 patch_size=4，則 N=12.25(非整數)
        → GraphConv → (B, N, gc_hidden)
        → 全域 average → (B, gc_hidden) → MLP Head → (B, num_classes)
        """
        B = x.size(0)

        # --- 1. CNN Stem ---
        f = self.stem(x)  # (B, hidden_dim, 14, 14)

        # --- 2. 切 patch: 當作圖上的 Node ---
        C, H, W = f.shape[1], f.shape[2], f.shape[3]
        p = self.patch_size
        assert H % p == 0 and W % p == 0, "patch_size 必須整除特徵圖大小"

        # 把 f reshape 成 [B, N, C, p, p]，再做平均: (p,p) → 1
        f = f.unfold(2, p, p).unfold(3, p, p)
        # f: (B, C, H/p, p, W/p, p) → transpose / reshape
        f = f.contiguous().view(B, C, H//p, p, W//p, p)
        # 先把最後兩個 p×p 維度壓掉：patch_avg = avg_pool  → (B, C, H/p, W/p)
        patch_avg = f.mean(dim=3).mean(dim=4)  # (B, C, H/p, W/p)

        # 再展平成 (B, N, C)，其中 N = (H/p)*(W/p)
        N = (H // p) * (W // p)
        patch_feat = patch_avg.view(B, C, N).permute(0, 2, 1)  # (B, N, C)

        # --- 3. 純手刻 GraphConv  ---
        #   構建 patch 間的 8-neighbor adjacency index
        #   先把 (B, N, C) 轉成格點 (B, C, H/p, W/p)，再對每個 node 蒐集 8 鄰域 feature
        h = H // p
        w = W // p  # h = w = 14/p (p=2 → h=w=7)

        # reshape 回 2D Grid
        grid = patch_feat.view(B, h, w, C)  # (B, h, w, C)

        # 先 pad 一圈 (value=0)，方便取鄰居
        pad_grid = F.pad(grid, (0, 0, 1, 1, 1, 1))  # pad top/bottom/left/right → (B, h+2, w+2, C)

        # 蒐集 8 鄰域：我們逐方向取 slice
        nei_sum = torch.zeros(B, h, w, C, device=x.device)
        for dy in [-1, 0, 1]:
            for dx in [-1, 0, 1]:
                if dy == 0 and dx == 0:
                    continue
                nei = pad_grid[:, 1+dy : 1+dy+h, 1+dx : 1+dx+w, :]  # (B, h, w, C)
                nei_sum += nei
        # 平均化（每個 node 最多有 8 個鄰居；邊界少一點，這裡直接除 8 保持簡單）
        nei_mean = nei_sum / 8.0  # (B, h, w, C)

        # 把自己 & 鄰居 feature 都做一次線性層：
        self_feat = self.gc_in(patch_feat)   # (B, N, gc_hidden)
        nei_feat  = self.gc_nei(nei_mean.view(B, N, C))  # (B, N, gc_hidden)
        gconv = self.gc_act(self_feat + nei_feat)  # (B, N, gc_hidden)

        # --- 4. 全球平均 + MLP Head ---
        # 先 (B, N, gc_hidden) → (B, gc_hidden) by 平均 over N
        graph_feat = gconv.mean(dim=1)  # (B, gc_hidden)

        logits = self.mlp_head(graph_feat)  # (B, num_classes)
        return logits



In [9]:
# Step 5: 初始化模型與優化器

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

model = PatchGNNNet(
    num_classes=len(train_dataset.classes),  # 22
    in_ch=3,                       # 輸入三通道 RGB
    stem_channels=64,              # CNN Stem 第一層輸出 64
    patch_size=2,                  # 把特徵圖 14×14 切成 2×2 → 7×7=49 nodes
    hidden_dim=128,                # CNN Stem 最後輸出通道 128 → patch feature dim
    gc_hidden=256                  # GNN 隱藏層維度 256
).to(device)


criterion = DistancePenaltyLoss(
    node_distance_matrix.to(device),
    area_distance_matrix.to(device),
    node_to_area.to(device),
    alpha=loss_alpha,
    beta=loss_beta
)

optimizer = optim.AdamW(model.parameters(), lr=1e-4)
scaler = GradScaler()


<ipython-input-9-1734f8dfd10f>:24: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [10]:
def train_one_epoch_amp(model, loader):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for batch_idx, (images, labels) in enumerate(loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        with torch.autocast(device.type if device.type != "mps" else "cpu", enabled=(device.type != "cpu")):
            outputs = model(images)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        # Print batch details
        print(f"Batch {batch_idx+1}/{len(loader)} | Loss: {loss.item():.4f} | Batch Acc: {(predicted == labels).float().mean().item():.4f}")
    acc = correct / total
    print(f"Epoch Train Loss: {total_loss:.4f}, Accuracy: {acc:.4f}")

def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    acc = correct / total
    print(f"Validation Accuracy: {acc:.4f}")
    return acc

In [11]:
# Step 7: 訓練流程
best_acc = 0.0

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    train_one_epoch_amp(model, train_loader)
    val_acc = evaluate(model, val_loader)
    # print("validation accuracy: ", val_acc)
    if val_acc > best_acc:
        best_acc = val_acc
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth")
        print(f"✅ 儲存最佳模型（Acc: {best_acc:.4f}）")
    else:
        patience_counter += 1
        if patience_counter >= early_stop_patience:
            print("⏹️ 觸發 Early Stopping")
            break

test_acc = evaluate(model, test_loader)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")


Epoch 1/50
Batch 1/72 | Loss: 7.1431 | Batch Acc: 0.0000
Batch 2/72 | Loss: 7.3595 | Batch Acc: 0.0000
Batch 3/72 | Loss: 7.5631 | Batch Acc: 0.0000
Batch 4/72 | Loss: 7.3832 | Batch Acc: 0.0625
Batch 5/72 | Loss: 7.5407 | Batch Acc: 0.0000
Batch 6/72 | Loss: 7.5120 | Batch Acc: 0.0000
Batch 7/72 | Loss: 7.5102 | Batch Acc: 0.0625
Batch 8/72 | Loss: 7.2032 | Batch Acc: 0.1250
Batch 9/72 | Loss: 7.5139 | Batch Acc: 0.0000
Batch 10/72 | Loss: 7.4157 | Batch Acc: 0.0625
Batch 11/72 | Loss: 7.4605 | Batch Acc: 0.0625
Batch 12/72 | Loss: 7.2414 | Batch Acc: 0.0625
Batch 13/72 | Loss: 7.5600 | Batch Acc: 0.0000
Batch 14/72 | Loss: 7.0495 | Batch Acc: 0.1875
Batch 15/72 | Loss: 7.7380 | Batch Acc: 0.0000
Batch 16/72 | Loss: 7.6669 | Batch Acc: 0.0000
Batch 17/72 | Loss: 7.5044 | Batch Acc: 0.0000
Batch 18/72 | Loss: 7.3517 | Batch Acc: 0.0625
Batch 19/72 | Loss: 7.1604 | Batch Acc: 0.2500
Batch 20/72 | Loss: 7.4905 | Batch Acc: 0.0000
Batch 21/72 | Loss: 7.3923 | Batch Acc: 0.1250
Batch 22/7

In [12]:
test_acc = evaluate(model, test_loader)
print(f"\nFinal Test Accuracy: {test_acc:.4f}")

Validation Accuracy: 0.3295

Final Test Accuracy: 0.3295


In [13]:
# Step 8: 載入最佳模型（推論前）
model.load_state_dict(torch.load("best_model.pth"))
model.to(device)

PatchGNNNet(
  (stem): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): Mish()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Mish()
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Mish()
    (10): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Mish()
  )
  (gc_in): Linear(in_features=128, out_features=256, bias=False)
  (gc_nei): Linear(in_features=128

In [14]:
from PIL import Image

# Load model
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
model.to(device)

# Load and preprocess image
img_path = "/content/drive/MyDrive/DLA_term_project_data/classified_data/test/node_15/IMG_4393.jpg"
img = Image.open(img_path).convert("RGB")
input_tensor = test_transform(img).unsqueeze(0).to(device)

# Predict
with torch.no_grad():
    logits = model(input_tensor)
    probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

# Print probabilities
for idx, prob in enumerate(probs):
    print(f"Node {idx+1}: {prob:.4f}")

Node 1: 0.0444
Node 2: 0.1711
Node 3: 0.0841
Node 4: 0.0410
Node 5: 0.0562
Node 6: 0.0062
Node 7: 0.1582
Node 8: 0.0930
Node 9: 0.0484
Node 10: 0.0333
Node 11: 0.0445
Node 12: 0.0031
Node 13: 0.0734
Node 14: 0.0353
Node 15: 0.0077
Node 16: 0.0038
Node 17: 0.0027
Node 18: 0.0017
Node 19: 0.0045
Node 20: 0.0068
Node 21: 0.0030
Node 22: 0.0776


In [15]:
import numpy as np
from collections import Counter

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print predictions and ground truths
for i, (pred, label) in enumerate(zip(all_preds, all_labels)):
    correct = "✅" if pred == label else "❌"
    print(f"Sample {i+1}: True Node={label+1}, Predicted Node={pred+1} {correct}")

# Count misclassifications per node
misclass_counts = Counter()
for pred, label in zip(all_preds, all_labels):
    if pred != label:
        misclass_counts[label+1] += 1  # +1 for 1-based node index

print("\nMisclassification count per node (True label):")
for node in range(1, len(test_loader.dataset.classes)+1):
    print(f"Node {node}: {misclass_counts[node]}")

# Find the most misclassified node(s)
if misclass_counts:
    most_misclassified = misclass_counts.most_common(1)[0]
    print(f"\nNode most often misclassified: Node {most_misclassified[0]} ({most_misclassified[1]} times)")
else:
    print("\nNo misclassifications!")

Sample 1: True Node=1, Predicted Node=14 ❌
Sample 2: True Node=1, Predicted Node=2 ❌
Sample 3: True Node=1, Predicted Node=14 ❌
Sample 4: True Node=1, Predicted Node=16 ❌
Sample 5: True Node=1, Predicted Node=14 ❌
Sample 6: True Node=1, Predicted Node=14 ❌
Sample 7: True Node=1, Predicted Node=14 ❌
Sample 8: True Node=1, Predicted Node=14 ❌
Sample 9: True Node=2, Predicted Node=2 ✅
Sample 10: True Node=2, Predicted Node=2 ✅
Sample 11: True Node=2, Predicted Node=19 ❌
Sample 12: True Node=2, Predicted Node=20 ❌
Sample 13: True Node=2, Predicted Node=2 ✅
Sample 14: True Node=2, Predicted Node=2 ✅
Sample 15: True Node=2, Predicted Node=2 ✅
Sample 16: True Node=2, Predicted Node=2 ✅
Sample 17: True Node=3, Predicted Node=2 ❌
Sample 18: True Node=3, Predicted Node=8 ❌
Sample 19: True Node=3, Predicted Node=19 ❌
Sample 20: True Node=3, Predicted Node=15 ❌
Sample 21: True Node=3, Predicted Node=9 ❌
Sample 22: True Node=3, Predicted Node=5 ❌
Sample 23: True Node=3, Predicted Node=8 ❌
Sample 24